In [80]:
#import необходимых модулей
!pip install category_encoders
from category_encoders.binary import BinaryEncoder

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, f1_score

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [81]:
#Считывание данных в DataFrame 

train = pd.read_csv('https://vk.com/doc3811482_641100498?hash=rtNJZfVDvpMYyZhiTRAQXIahyqn70z1Ot8fnVBjINzL&dl=a4WUw6niZ5v7TmWg40GnRKHJHczdsMZxx0GuBSWo9vX', sep=';')
test = pd.read_csv('https://vk.com/doc3811482_641224869?hash=28p1Kn2R30tLn8WXYjNqtrZ14oLXoAJdLGpOvGT2JN8&dl=JZrmo9xBoyw0zLrufhvW5XjN9UP2N2CfKab0DgSUjqk', sep=';')
testID = test.id 
test = test.drop(['id'], axis = 1)

test

In [82]:
train = pd.concat([train,test],ignore_index = True)
train = train.fillna(0)
train

In [83]:
def region_1(x): # первая функция для региона
    if 'г.' in x.lower():
        return x.lower().replace('г. ', '').strip()
    if 'авт. область' in x.lower():
        return x.lower().replace('авт. область', '').strip()
    if 'область' in x.lower():
        return x.lower().replace('область', '').strip()
    if 'республика' in x.lower():
        return x.lower().replace('республика', '').strip()
    if 'авт. округ' in x.lower():
        return x.lower().replace('авт. округ', '').strip()
    if 'край' in x.lower():
        return x.lower().replace('край', '').strip()                              
    else:
        return x.lower()
    
def region_2(x): # вторая функция для региона
    x = x.replace(',', '')
    return x.split()[0]  

In [84]:
train['region'] = train['subject_name'].map(region_1).map(region_2)

zarplata = pd.read_csv('https://vk.com/doc3811482_641133658?hash=bRcG4ZqY2Y2YkCiUfgrH1PTq5EZe1pU3qYLqqhEIQLX&dl=ekJAkq44FIg0W6n8BM72SfTJKl2JHQg15VPKvGXGcKo')
train = train.merge(zarplata, on='region')

population = pd.read_csv('https://vk.com/doc3811482_641134413?hash=EGySA0TaBzseOiikK1icTH3aXHRd6ZrXhZFEDPaXOF0&dl=DTgPEfP7DckJ8q52pZTr4lOd1hNzUGpa9vTWWq8bTFc')
train = train.merge(population, on='region')

zapros = pd.read_csv('https://vk.com/doc3811482_641171997?hash=tDkwknZczHKQ6SUzqBG7cJt2IlzJcaO8r4JYWtNJ3BP&dl=ghomlk37IqRZ9JED9aZ2fdq2V7Ke23NGUAEMNZQOWRw')
train['city'] = train['city_name'].str.lower()
train = train.merge(zapros, on='city', how='left')
train.rostelecom_tarif_yandex = train.rostelecom_tarif_yandex.fillna(0)

oborot_rozn = pd.read_csv('https://vk.com/doc3811482_641172075?hash=8MYO0eZ9Xu9LaP1Z2oXQh3UnWslZjyIHSNiPjzUjfM4&dl=ON5QTA8omF0cRa7IdbDVJ2GAcAeaRK7M2iYZVpuhQZs')
train = train.merge(oborot_rozn, on='region', how='left')

tarif_game = pd.read_csv('https://vk.com/doc3811482_641172128?hash=OR6ruaFueylo8QgBvCGh8aZNwezaHbxZYsszO5zRSPc&dl=mYL1vrozRJackj755F6wii7LIt58El56dnaQyNxEdL4')
train = train.merge(tarif_game, on='city', how='left')
train.tarif_game = train.tarif_game.fillna(0)

rostelecom = pd.read_csv('https://vk.com/doc3811482_641172201?hash=57GPGHaJDoRvJuCLe394WbtQdBk5H2hinrzUsZ8ihJH&dl=86MfGagB1Z3NsapjgIgvPZNzb10tV2l3H5ZjnYxwjMc')
train = train.merge(rostelecom, on='city', how='left')
train.rostelecom = train.rostelecom.fillna(0)

train


In [85]:
train['period'] = pd.to_datetime(train['period'])
# train['year'] = pd.DatetimeIndex(train['period']).year
train['mounth'] = pd.DatetimeIndex(train['period']).month 
# train['mounth_por'] = (train.year- 2020)*12 + train.mounth
train['summer'] = train['mounth'].map(lambda x: 1 if x in [6, 7, 8] else 0)
train['winter'] = train['mounth'].map(lambda x: 1 if x in [12, 1, 2] else 0)
train['spring-autumn'] = train['mounth'].map(lambda x: 1 if x in [3, 4, 5, 9, 10, 11] else 0)
train = train.drop(['period','mounth'], axis = 1)

print(train.winter.sum())
print(train.summer.sum())

In [86]:
test2 = train[294258:]
y_test2_label = test2.label

train = train[:294258]
train

In [87]:
train.city_name = train.city_name.astype('category')
test2.city_name = test2.city_name.astype('category')
bn = BinaryEncoder()
train = pd.concat([train, bn.fit_transform(train.city_name)], axis = 1)
test2 = pd.concat([test2, bn.transform(test2.city_name)], axis = 1)

train.hex = train.hex.astype('category')
test2.hex = test2.hex.astype('category')
bn2 = BinaryEncoder()
train = pd.concat([train, bn2.fit_transform(train.hex)], axis = 1)
test2 = pd.concat([test2, bn2.transform(test2.hex)], axis = 1)

test2

In [88]:
train = train.drop(columns=['subject_name','city_name','hex','subject_type','region','city'], axis = 1)
test2 = test2.drop(columns=['subject_name','city_name','hex','subject_type','region','city'], axis = 1) #'period'

train

In [89]:
X = train.drop(['label'], axis = 1)
y = train[['label']]

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)
X_train

In [91]:
y_train1 = y_train.to_numpy()
y_test1 = y_test.to_numpy()
y_test2_real = y_test2_label.to_numpy() #значения на нашем тесте
y_train1

In [92]:
#Исправляем дисбаланс классов - не работает
# from imblearn.under_sampling import NearMiss
# nm = NearMiss()
# X_train_miss, y_train_miss = nm.fit_resample(X_train, y_train1.ravel())

# print('После применения метода кол-во меток со значением True: {}'.format(sum(y_train_miss == 1)))
# print('После применения метода кол-во меток со значением False: {}'.format(sum(y_train_miss == 0)))

In [93]:
# from imblearn.over_sampling import SMOTE
# from collections import Counter
# X_train, y_train1 = SMOTE().fit_resample(X_train, y_train1)
# print(sorted(Counter(y_train1).items()))


# Catboost

In [94]:
#Установка catboost
!pip install catboost

In [95]:
pool_train = Pool(X_train, y_train1)
pool_test = Pool(X_test)
pool_test_real = Pool(test2)

In [96]:
    model2 = CatBoostClassifier(task_type='GPU',iterations = 2000, loss_function = 'MultiClassOneVsAll') #,iterations = 2000, learning_rate=0.11,
                        #depth=10,l2_leaf_reg=0.001,rsm=None,loss_function='RMSE'
    model2.fit(pool_train,plot=True)

    y_pred = model2.predict(pool_test)
    print("Значение метрики Precision на test: ", precision_score(y_test, y_pred))
    print("Значение метрики F1_score на test: ", f1_score(y_test, y_pred))

In [97]:
importances = model2.feature_importances_

feature_list = list(X.columns)

# Extract the feature importances into a dataframe
feature_results = pd.DataFrame({'feature': feature_list,'importance': importances})

# Show the top 10 most important
feature_results = feature_results.sort_values('importance',ascending = False).reset_index(drop=True)

feature_results.head(10)

In [98]:
y_pred = model2.predict(pool_test)
print("Значение метрики Precision на test: ", precision_score(y_test, y_pred))
print("Значение метрики F1_score на test: ", f1_score(y_test, y_pred))
#0.4625 precision

In [99]:
y_pred.sum()

**Предсказываем тест**

In [100]:
y_pred_real = model2.predict(pool_test_real)
# print("Значение метрики Precision на test: ", precision_score(y_test2_real, y_pred_real)) #0.35
# print("Значение метрики F1_score на test: ", f1_score(y_test2_real, y_pred_real))

In [101]:
y_pred_real.sum()

In [102]:
#Увеличиваем долю едениц искуственно за счет предсказаний с меньшей долей вероятности
y_pred_real_proba = model2.predict_proba(pool_test_real)
y_pred_real_proba2 = []
y_pred_real_proba_rezult = []
for i in range(len(y_pred_real_proba)):
    y_pred_real_proba_rezult.append(y_pred_real_proba[i][1])
    if y_pred_real_proba[i][1] > 0.5:
        y_pred_real_proba2.append(1)
    else: y_pred_real_proba2.append(0)
y_pred_real_proba2 = np.asarray(y_pred_real_proba2)
y_pred_real_proba_rezult = np.asarray(y_pred_real_proba_rezult)
print("Количество предсказанных покупок: ", y_pred_real_proba2.sum())
print("Значение метрики Precision на test: ", precision_score(y_test2_real, y_pred_real_proba2))
print("Значение метрики F1_score на test: ", f1_score(y_test2_real, y_pred_real_proba2))

# Precision на топ-5

In [103]:
data = {'id':testID, 'label': y_pred_real_proba_rezult, 'label2': y_pred_real_proba2, 'real10': y_test2_real}

df_otvet = pd.DataFrame(data)
df_otvet = df_otvet.sort_values('label', ascending = False)

# df_otvet = df_otvet[0 : int(round(y_pred_real_proba.shape[0]*0.05,0))]

# df_otvet[0 : int(round(y_pred_real_proba.shape[0]*0.05,0))].label2 = 1
df_otvet



In [104]:
df_otvet = df_otvet.drop(['label2','real10'], axis = 1)

df_otvet = df_otvet[0 : int(round(y_pred_real_proba.shape[0]*0.05,0))]
df_otvet

In [105]:
df_otvet.to_csv('solution.csv', sep=';', index=None)